# Exploitation Sprint: Unified Experiment Runner
**Goal**: Run any experiment config with 3 seeds, evaluate, save.

**Usage**: Change `CONFIG` in Cell 2 to run different experiments:
- `configs/gru_swa_vanilla_v1.yaml` (Day 1: SWA)
- `configs/gru_featdrop_A.yaml` (Day 2: drop p6-p11, 26 features)
- `configs/gru_featdrop_B.yaml` (Day 2: drop p6-p11+v6-v11, 20 features)
- `configs/gru_wd1e5_v1.yaml` (Day 3: weight decay 1e-5)
- `configs/gru_gradclip05_v1.yaml` (Day 3: gradient clip 0.5)
- `configs/gru_bs128_v1.yaml` (Day 3: batch size 128)

**Kill gate**: mean val (3 seeds) > 0.2709

In [ ]:
# Cell 1: Mount Drive, download data from Kaggle, clone repo
import os, json, subprocess

from google.colab import drive
drive.mount('/content/drive')
os.makedirs('/content/drive/MyDrive/wunderfund', exist_ok=True)

!pip install -q kaggle==1.6.14 --force-reinstall
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump({"username": "vincentvdo6", "key": "FILL_IN"}, f)
os.chmod('/root/.kaggle/kaggle.json', 0o600)

os.makedirs('/content/data', exist_ok=True)
!kaggle datasets download -d vincentvdo6/wunderfund-predictorium -p /content/data/ --force
!unzip -o -q /content/data/wunderfund-predictorium.zip -d /content/data/
!ls /content/data/*.parquet

# Clone repo
REPO = "/content/competition_package"
os.chdir("/content")
subprocess.run(["rm", "-rf", REPO], check=False)
subprocess.run(["git", "clone", "https://github.com/vincentvdo6/competition_package.git", REPO], check=True)
os.chdir(REPO)
os.makedirs("datasets", exist_ok=True)
os.makedirs("logs", exist_ok=True)

subprocess.run(["ln", "-sf", "/content/data/train.parquet", "datasets/train.parquet"], check=True)
subprocess.run(["ln", "-sf", "/content/data/valid.parquet", "datasets/valid.parquet"], check=True)

assert os.path.exists("datasets/train.parquet"), "train.parquet not found!"
assert os.path.exists("datasets/valid.parquet"), "valid.parquet not found!"

commit = subprocess.check_output(["git", "rev-parse", "--short", "HEAD"], text=True).strip()
print(f"Commit: {commit}")
print(f"GPU: {subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], text=True).strip()}")
print("Ready!")

In [ ]:
# Cell 2: === CHANGE THIS CONFIG FOR EACH EXPERIMENT ===
CONFIG = "configs/gru_swa_vanilla_v1.yaml"  # <-- CHANGE ME
SEEDS = [42, 43, 44]

# Derive experiment name from config
import os
EXP_NAME = os.path.splitext(os.path.basename(CONFIG))[0]
print(f"Experiment: {EXP_NAME}")
print(f"Config: {CONFIG}")
print(f"Seeds: {SEEDS}")

In [ ]:
# Cell 3: Train all seeds (skip existing)
import subprocess, sys, os
os.chdir("/content/competition_package")

print(f"=== {EXP_NAME.upper()} ({len(SEEDS)} seeds) ===")
print(f"Config: {CONFIG}")
print("=" * 60, flush=True)

for seed in SEEDS:
    ckpt = f"logs/{EXP_NAME}_seed{seed}.pt"
    if os.path.exists(ckpt):
        print(f"seed {seed}: checkpoint exists -- skip")
        continue
    print(f"\n{'='*60}")
    print(f"{EXP_NAME} seed {seed}")
    print(f"{'='*60}", flush=True)
    proc = subprocess.Popen(
        [sys.executable, "-u", "scripts/train.py",
         "--config", CONFIG,
         "--seed", str(seed), "--device", "cuda"],
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
    )
    for line in proc.stdout:
        print(line, end="", flush=True)
    rc = proc.wait()
    if rc != 0:
        print(f"ERROR: seed {seed} failed with rc={rc}")

print(f"\nTraining complete!")

In [ ]:
# Cell 4: Evaluate all seeds
import os, torch
os.chdir("/content/competition_package")

BASELINE_VALS = {42: 0.2649, 43: 0.2737, 44: 0.2690}  # vanilla GRU baseline
GATE = 0.2709  # kill gate

print(f"{'Seed':<6} {EXP_NAME:>12} {'GRU Base':>10} {'Delta':>10}")
print("-" * 45)

scores = []
for seed in SEEDS:
    pt = f"logs/{EXP_NAME}_seed{seed}.pt"
    if not os.path.exists(pt):
        print(f"s{seed}:  MISSING")
        continue
    ckpt = torch.load(pt, map_location="cpu", weights_only=False)
    score = float(ckpt.get("best_score", 0))
    epoch = ckpt.get("best_epoch", "?")
    baseline = BASELINE_VALS.get(seed, 0)
    delta = score - baseline
    scores.append(score)
    print(f"s{seed:<5} {score:>12.4f} {baseline:>10.4f} {delta:>+10.4f}  (ep {epoch})")

if scores:
    mean_val = sum(scores) / len(scores)
    mean_base = sum(BASELINE_VALS[s] for s in SEEDS if s in BASELINE_VALS) / len(SEEDS)
    mean_delta = mean_val - mean_base
    print(f"\nMean {EXP_NAME}:  {mean_val:.4f}")
    print(f"Mean GRU base:    {mean_base:.4f}")
    print(f"Mean delta:       {mean_delta:+.4f}")
    print(f"\nKill gate ({GATE:.4f}): {'PASS' if mean_val > GATE else 'FAIL'}")
    if mean_val > GATE:
        print("-> Scale to 10 seeds + submit canary!")
    elif mean_val > 0.2695:
        print("-> Near miss. Train 5 more seeds to check.")
    else:
        print(f"-> KILL {EXP_NAME}.")

In [ ]:
# Cell 5: Save checkpoints to Drive + zip for download
import os, torch, shutil, glob, zipfile
os.chdir("/content/competition_package")

DRIVE_DIR = "/content/drive/MyDrive/wunderfund"

# Save slim checkpoints to Drive
for pt in sorted(glob.glob(f"logs/{EXP_NAME}_seed*.pt")):
    if '_epoch' in pt:
        continue
    seed = os.path.basename(pt).replace('.pt', '').split('seed')[-1]
    ckpt = torch.load(pt, map_location="cpu", weights_only=False)
    slim = {
        "model_state_dict": ckpt["model_state_dict"],
        "config": ckpt.get("config", {}),
        "best_score": ckpt.get("best_score", None),
        "best_epoch": ckpt.get("best_epoch", None),
    }
    dst = f"{DRIVE_DIR}/{EXP_NAME}_seed{seed}.pt"
    torch.save(slim, dst)
    sz = os.path.getsize(dst) / 1e6
    print(f"Saved: {EXP_NAME}_seed{seed}.pt ({sz:.1f}MB)")

# Also zip for quick download
zip_name = f"/content/{EXP_NAME}_checkpoints.zip"
with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zf:
    for pt in sorted(glob.glob(f"logs/{EXP_NAME}_seed*.pt")):
        if '_epoch' in pt:
            continue
        zf.write(pt, os.path.basename(pt))
sz = os.path.getsize(zip_name) / 1e6
print(f"\nZip: {zip_name} ({sz:.1f}MB)")
print("Done!")